In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
feature_name_df = pd.read_csv('./datasets/UCI HAR Dataset/features.txt', sep='\s+', header=None, names=['columns_index', 'column_name'])

feature_name = feature_name_df.iloc[:, 1].values.tolist()
print(f'전체 피처명에서 10개만 추출: {feature_name[:10]}')

전체 피처명에서 10개만 추출: ['tBodyAcc-mean()-X', 'tBodyAcc-mean()-Y', 'tBodyAcc-mean()-Z', 'tBodyAcc-std()-X', 'tBodyAcc-std()-Y', 'tBodyAcc-std()-Z', 'tBodyAcc-mad()-X', 'tBodyAcc-mad()-Y', 'tBodyAcc-mad()-Z', 'tBodyAcc-max()-X']


In [8]:
def get_new_feature_name_df(old_feature_name_df):
    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(), columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x : x[0] + '_' + str(x[1])
                                                                                              if x[1] > 0 else x[0], axis=1)
    
    new_feature_name_df = new_feature_name_df.drop(['index'], axis=1)
    return new_feature_name_df

In [9]:
import pandas as pd

def get_human_datasets():
    feature_name_df = pd.read_csv('./datasets/UCI HAR Dataset/features.txt', sep='\s+',
                                 header=None, names=['column_index', 'column_name'])
    
    new_feature_name = get_new_feature_name_df(feature_name_df)
#     print(new_feature_name)
#     print(new_feature_name[new_feature_name.duplicated(['column_name'])])
    feature_name  = new_feature_name.iloc[:, 1].values.tolist()
    X_train = pd.read_csv('./datasets/UCI HAR Dataset/train/X_train.txt', sep='\s+',  names=feature_name)
    X_test = pd.read_csv('./datasets/UCI HAR Dataset/test/X_test.txt', sep='\s+', names=feature_name)
    
    y_train = pd.read_csv('./datasets/UCI HAR Dataset/train/y_train.txt', sep='\s+', header=None, names=['action'])
    y_test = pd.read_csv('./datasets/UCI HAR Dataset/test/y_test.txt', sep='\s+', header=None, names=['action'])
    
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = get_human_datasets()

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

dtree = DecisionTreeClassifier(random_state=156)
dtree.fit(X_train, y_train)
pred = dtree.predict(X_test)
score = accuracy_score(y_test, pred)
print(score)

# score_df = pd.DataFrame(grid_dtree.cv_results_)

0.8547675602307431


In [12]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

params = {
    'max_depth' : [8, 12, 16, 20],
    'min_samples_split' : [16, 24]
}

grid_cv = GridSearchCV(dtree, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(X_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
0.8548794147162603
{'max_depth': 8, 'min_samples_split': 16}


In [8]:
best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(X_test)
score = accuracy_score(y_test, pred1)
print(score)

0.8717339667458432


In [9]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(random_state=42)
lg.fit(X_train, y_train)
pred = lg.predict(X_test)
accuracy_score(y_test, pred)

0.9579233118425518

In [10]:
from sklearn.svm import SVC
lg = SVC(random_state=42)
lg.fit(X_train, y_train)
pred = lg.predict(X_test)
accuracy_score(y_test, pred)

0.9504580929759077

In [11]:
from sklearn.ensemble import RandomForestClassifier
lg = RandomForestClassifier(random_state=42)
lg.fit(X_train, y_train)
pred = lg.predict(X_test)
accuracy_score(y_test, pred)

0.9256871394638616